# T5 model for text simplification

## import package

In [1]:
!pip install --upgrade pip
!pip install --upgrade torch
!pip install SentencePiece
!pip uninstall transformers -y
!pip install transformers
!pip install rich
!pip install rouge
!pip install evaluate
!pip install Levenshtein

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Found existing installation: transformers 4.28.1
Uninstalling transformers-4.28.1:
  Successfully uninstalled transformers-4.28.1
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d8/a7/a6ff727fd5d96d6625f4658944a2ae230f0c75743a9a117fbda013b03d3d/transformers-4.28.1-py3-none-any.whl (7.0 MB)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
import string
import json
import pandas as pd
from tqdm import tqdm
import re
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5TokenizerFast, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console
from torch import cuda

## Train data

In [4]:
def load_data(sou_path):
    with open(sou_path, 'r', encoding='utf-8') as sou_file:
                # 循环读取源文件数据，并写入到目标文件中
        file = json.load(sou_file)
        return file

In [5]:
current_path = os.getcwd()
print("Current path:", current_path)

Current path: D:\Program Files (x86)\Anaconda3\NLP project


In [6]:
file1=load_data('train/simpletext_task3_train.json')
file2=load_data('train/simpletext_task3_qrels.json')

In [7]:
query_lst=[]
for data in file1:
    query_lst.append(data['query_text'])
print(f"Data size: {len(file1)}")
query_dict={i:query_lst.count(i) for i in query_lst}
print(f"The number of query text: {len(query_dict)}")
print(f"The amount of data per query text: {query_dict}")

Data size: 648
The number of query text: 43
The amount of data per query text: {'drones': 31, 'self driving': 132, 'misinformation': 63, 'cryptocurrency': 41, 'forensics': 54, 'Digital assistant': 2, 'Biases': 6, 'humanoid robots': 34, 'digital marketing': 13, 'Privacy': 7, 'smart speaker': 17, 'drug discovery': 8, 'functional genomics': 3, 'infectious diseases': 5, 'gene editing': 20, 'crispr': 4, 'conspiracy theories': 18, 'financial markets': 11, 'Alcohol interfer with recovery and adaptation to training': 32, 'The problem Of muscle hypertrophy: Revisited': 5, 'Occlusion Training for Hypertrophy': 5, 'Hypertrophy after Aerobic Exercise': 3, 'Stretch training induces hypertrophy?': 7, 'Cycle training induces muscle hypertrophy': 7, 'Volume for hypertrophy': 7, 'short versus long inter-set rest intervals on hypertrophy': 4, 'Muscle memory': 9, 'Effects of meal frequency on weight loss': 6, 'Resistance Training is Medicine': 7, 'Benefits of strength training for adolescents': 7, 'Resis

In [8]:
df1=pd.DataFrame(file1,dtype='object')
df1['source_snt'] = df1['source_snt'].str.replace('.',',')
df1['source_snt'] = df1[['source_snt', 'query_text']].agg(' related to '.join, axis=1)+"."
df2=pd.DataFrame(file2)
df_total=pd.concat([df1,df2],axis=1)

<ipython-input-8-955af95913bf>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df1['source_snt'] = df1['source_snt'].str.replace('.',',')


In [9]:
df=df_total[['source_snt','simplified_snt']]
df

,source_snt,simplified_snt
0,"In the modern era of automation and robotics, ...",Current academic and industrial research is in...
1,With the ever increasing number of unmanned ae...,Drones are increasingly used in the civilian a...
2,Due to guidelines set by the governments regar...,Governments set guidelines on the operation ce...
3,In an attempt to achieve the above mentioned t...,Researchers propose data-driven solutions allo...
4,Derived from the classic image classification ...,"The algorithm, based on the Inception model, d..."
...,...,...
643,Bodybuilders generally train with moderate loa...,Bodybuilders train with moderate loads and sho...
644,"Powerlifters, on the other hand, routinely tra...","Powerlifters, on the other hand, train with hi..."
645,Although both groups are known to display impr...,Although both groups are known to display impr...
646,It has been shown that many factors mediate th...,It has been shown that many factors mediate th...


In [10]:
def data_info(data,column_name):
    max_length=max([len(i.split()) for i in data[column_name]])
    average_length=sum([len(i.split()) for i in data[column_name]])/len(data[column_name])
    return max_length,average_length

max_length,average_length=data_info(df,'source_snt')
print({'Dataframe':'Train data','max length of source sentence':max_length,'avg length of source sentence':average_length})

max_length,average_length=data_info(df,'simplified_snt')
print({'Dataframe':'Train data','max length of simplified sentence':max_length,'avg length of simplified sentence':average_length})

{'Dataframe': 'Train data', 'max length of source sentence': 71, 'avg length of source sentence': 28.734567901234566}
{'Dataframe': 'Train data', 'max length of simplified sentence': 61, 'avg length of simplified sentence': 18.814814814814813}


 ## Add control tokens to training data

In [2]:
from string import punctuation
import Levenshtein
import spacy
import nltk
import time
from nltk.corpus import stopwords
import multiprocessing
from multiprocessing import Pool, Lock
from tqdm import tqdm
nltk.download('stopwords')
import threading
from queue import Queue
from functools import lru_cache

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Crystal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
from source.resources import DUMPS_DIR, WORD_FREQUENCY_FILEPATH, WORD_EMBEDDINGS_NAME, download_glove,COMPLEXITY_MODEL_FILEPATH,GOOGLE_WORD_EMBEDDINGS_FILEPATH
from source.helper import tokenize, yield_lines, load_dump, dump, write_lines,save_preprocessor, yield_sentence_pair,generate_hash

In [11]:
stopwords = set(stopwords.words('english'))

def round(val):
    return '%.2f' % val

def safe_division(a, b):
    return a / b if b else 0

@lru_cache(maxsize=1024)
def is_punctuation(word):
    return ''.join([char for char in word if char not in punctuation]) == ''

def remove_punctuation(text):
    return ' '.join([word for word in tokenize(text) if not is_punctuation(word)])

def remove_stopwords(text):
    return ' '.join([w for w in tokenize(text) if w.lower() not in stopwords])

In [12]:
@lru_cache(maxsize=1)
def get_spacy_model():
    model = 'en_core_web_sm'
    if not spacy.util.is_package(model):
        spacy.cli.download(model)
        spacy.cli.link(model, model, force=True, model_path=spacy.util.get_package_path(model))
    return spacy.load(model)

@lru_cache(maxsize=10 ** 6)
def spacy_process(text):
    return get_spacy_model()(str(text))

In [13]:
@lru_cache(maxsize=1024)
def get_dependency_tree_depth(sentence):
    def tree_height(node):
        if len(list(node.children)) == 0:
            return 0
        return 1 + max(tree_height(child) for child in node.children)

    tree_depths = [tree_height(spacy_sentence.root) for spacy_sentence in spacy_process(sentence).sents]
    if len(tree_depths) == 0:
        return 0
    return max(tree_depths)

In [14]:
@lru_cache(maxsize=1)
def get_word2rank(vocab_size=np.inf):
    model_filepath = DUMPS_DIR / f"{WORD_EMBEDDINGS_NAME}.pk"
    if model_filepath.exists():
        return load_dump(model_filepath)
    else:
        print("Downloading glove.42B.300d ...")
        download_glove(model_name='glove.42B.300d', dest_dir=str(DUMPS_DIR))
        print("Preprocessing word2rank...")
        DUMPS_DIR.mkdir(parents=True, exist_ok=True)
        WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
        lines_generator = yield_lines(WORD_EMBEDDINGS_PATH)
        word2rank = {}
        # next(lines_generator)
        for i, line in enumerate(lines_generator):
            if i >= vocab_size: break
            word = line.split(' ')[0]
            word2rank[word] = i
        dump(word2rank, model_filepath)
        txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
        zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
        if txt_file.exists(): txt_file.unlink()
        if zip_file.exists(): zip_file.unlink()
        return word2rank

get_word2rank()

@lru_cache(maxsize=10000)
def get_normalized_rank(word):
    max = len(get_word2rank())
    rank = get_word2rank().get(word, max)
    return np.log(1 + rank) / np.log(1 + max)

@lru_cache(maxsize=5000)
def get_rank(word):
    rank = get_word2rank().get(word, len(get_word2rank()))
    return np.log(1 + rank)

@lru_cache(maxsize=2048)
def get_lexical_complexity_score(sentence):
    words = tokenize(remove_stopwords(remove_punctuation(sentence)))
    words = [word for word in words if word in get_word2rank()]  # remove unknown words
    if len(words) == 0:
        return 1.0
#     return np.array([get_normalized_rank(word) for word in words]).mean()
    return np.quantile([get_rank(word) for word in words], 0.75)

# @lru_cache(maxsize=2048)
def get_lexical_complexity_score_batch(simple_sentences,complex_sentences):
    scores=[]
    for i in range(len(simple_sentences)):
        scores.append(float(round(min(safe_division(get_lexical_complexity_score(simple_sentences[i]),
                                       get_lexical_complexity_score(complex_sentences[i])), 2))))
    return np.mean(scores)

In [15]:
simple_sentences=['Current academic and industrial research is interested in autonomous vehicles.','Current academic and industrial research is interested in autonomous vehicles.']
complex_sentences=['In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.','In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.']
get_lexical_complexity_score_batch(simple_sentences,complex_sentences)

0.88

In [16]:

@lru_cache(maxsize=1)
def get_word_frequency():
    model_filepath = DUMPS_DIR / f'{WORD_FREQUENCY_FILEPATH.stem}.pk'
    if model_filepath.exists():
        return load_dump(model_filepath)
    else:
        DUMPS_DIR.mkdir(parents=True, exist_ok=True) 
        word_freq = {}
        with open(WORD_FREQUENCY_FILEPATH,'r',encoding='utf-8') as f:
            for line in f:
                chunks = line.rstrip().split(' ')
                word = chunks[0]
                freq = int(chunks[1])
                word_freq[word] = freq
        dump(word_freq, model_filepath)
        return word_freq

get_word_frequency()

@lru_cache(maxsize=10000)
def get_normalized_frequency(word):
    max = 179573112 # the 153141437, the max frequency
    freq = get_word_frequency().get(word, 0)
    return 1.0 - np.log(1 + freq) / np.log(1 + max)

@lru_cache(maxsize=2048)
def get_complexity_score(sentence):
    words = tokenize(remove_stopwords(remove_punctuation(sentence)))
#     words = tokenize(remove_punctuation(sentence))
    words = [word for word in words if word in get_word_frequency()]  # remove unknown words
    if len(words) == 0:
        return 1.0
    
    return np.array([get_normalized_frequency(word.lower()) for word in words]).mean()

In [17]:
sentence='Current academic and industrial research is interested in autonomous vehicles.'
# sentence='In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.'
words = tokenize(remove_stopwords(remove_punctuation(sentence)))
words = [word for word in words if word in get_word2rank()]
rank=np.quantile([get_rank(word) for word in words], 0.75)
# rank=np.array([get_normalized_rank(word) for word in words]).mean()
rank

7.837459915682472

In [18]:
class RatioFeature:
    def __init__(self, feature_extractor, target_ratio=0.80):
        self.feature_extractor = feature_extractor
        self.target_ratio = str(target_ratio)

    def encode_sentence(self, sentence):
        return f'{self.name}_{self.target_ratio}'

    def encode_sentence_pair(self, complex_sentence, simple_sentence):
        return f'{self.name}_{self.feature_extractor(complex_sentence, simple_sentence)}', simple_sentence

    def decode_sentence(self, encoded_sentence):
        return encoded_sentence

    @property
    def name(self):
        class_name = self.__class__.__name__.replace('RatioFeature', '')
        name = ""
        for word in re.findall('[A-Z][^A-Z]*', class_name):
            if word: name += word[0]
        if not name: name = class_name
        return name

In [19]:
class WordRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_word_length_ratio, *args, **kwargs)

    def get_word_length_ratio(self, complex_sentence, simple_sentence):
        return round(safe_division(len(tokenize(simple_sentence)), len(tokenize(complex_sentence))))

class CharRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_char_length_ratio, *args, **kwargs)

    def get_char_length_ratio(self, complex_sentence, simple_sentence):
        return round(safe_division(len(simple_sentence), len(complex_sentence)))


class LevenshteinRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_levenshtein_ratio, *args, **kwargs)

    def get_levenshtein_ratio(self, complex_sentence, simple_sentence):
        return round(Levenshtein.ratio(complex_sentence, simple_sentence))


class WordRankRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_word_rank_ratio, *args, **kwargs)

    def get_word_rank_ratio(self, complex_sentence, simple_sentence):
        return round(min(safe_division(self.get_lexical_complexity_score(simple_sentence),
                                       self.get_lexical_complexity_score(complex_sentence)), 2))

    def get_lexical_complexity_score(self, sentence):
        words = tokenize(remove_stopwords(remove_punctuation(sentence)))
        words = [word for word in words if word in get_word2rank()]
        if len(words) == 0:
            return np.log(1 + len(get_word2rank()))
        return np.quantile([self.get_rank(word) for word in words], 0.75)

    @lru_cache(maxsize=5000)
    def get_rank(self, word):
        rank = get_word2rank().get(word, len(get_word2rank()))
        return np.log(1 + rank)


class DependencyTreeDepthRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_dependency_tree_depth_ratio, *args, **kwargs)

    def get_dependency_tree_depth_ratio(self, complex_sentence, simple_sentence):
        return round(
            safe_division(self.get_dependency_tree_depth(simple_sentence),
                          self.get_dependency_tree_depth(complex_sentence)))
    
    @lru_cache(maxsize=1024)
    def get_dependency_tree_depth(self, sentence):
        def get_subtree_depth(node):
            if len(list(node.children)) == 0:
                return 0
            return 1 + max([get_subtree_depth(child) for child in node.children])

        tree_depths = [get_subtree_depth(spacy_sentence.root) for spacy_sentence in self.spacy_process(sentence).sents]
        if len(tree_depths) == 0:
            return 0
        return max(tree_depths)
    
    @lru_cache(maxsize=10 ** 6)
    def spacy_process(self, text):
        return get_spacy_model()(text)

In [20]:
class Preprocessor:
    def __init__(self, features_kwargs=None):
        super().__init__()

        self.features = self.get_features(features_kwargs)
        if features_kwargs:
            self.hash = generate_hash(str(features_kwargs).encode())
        else:
            self.hash = "no_feature"

    def get_class(self, class_name, *args, **kwargs):
        return globals()[class_name](*args, **kwargs)

    def get_features(self, feature_kwargs):
        features = []
        for feature_name, kwargs in feature_kwargs.items():
            features.append(self.get_class(feature_name, **kwargs))
        return features

    def encode_sentence(self, sentence):
        if self.features:
            line = ''
            for feature in self.features:
                line += feature.encode_sentence(sentence) + ' '
            line += sentence
            return line.rstrip()
        else:
            return sentence

    def encode_sentence_pair(self, complex_sentence, simple_sentence):
        if self.features:
            line = ''
            for feature in self.features:
                processed_complex, _ = feature.encode_sentence_pair(complex_sentence, simple_sentence)
                line += processed_complex + ' '
            line += complex_sentence
            return line.rstrip()

        else:
            return complex_sentence

    def decode_sentence(self, encoded_sentence):
        for feature in self.features:
            decoded_sentence = feature.decode_sentence(encoded_sentence)
        return decoded_sentence

    def encode_file(self, input_filepath, output_filepath):
        with open(output_filepath, 'w') as f:
            for line in yield_lines(input_filepath):
                f.write(self.encode_sentence(line) + '\n')

    def decode_file(self, input_filepath, output_filepath):
        with open(output_filepath, 'w') as f:
            for line in yield_lines(input_filepath):
                f.write(self.decode_sentence(line) + '\n')

    def encode_dataframe(self,dataset):
        processed_complex_sentences = []
        for complex_sentence, simple_sentence in tqdm(zip(dataset['source_snt'], dataset['simplified_snt']),total=len(dataset)):
            processed_complex_sentence = self.encode_sentence_pair(complex_sentence, simple_sentence)
            processed_complex_sentences.append(processed_complex_sentence)
        return processed_complex_sentences

    def preprocess_dataset(self, dataset):
        new_df=dataset.copy()
        new_df['source_snt']= self.encode_dataframe(dataset)
        return new_df

In [21]:
features_kwargs = {
        'WordRatioFeature': {'target_ratio': '1.05'},
        'CharRatioFeature': {'target_ratio': '0.95'},
        'LevenshteinRatioFeature': {'target_ratio': '0.75'},
        'WordRankRatioFeature': {'target_ratio': '0.85'},
        'DependencyTreeDepthRatioFeature': {'target_ratio': '0.85'}
    }
# features_kwargs = {}
preprocessor = Preprocessor(features_kwargs)

In [22]:
preprocessor = Preprocessor(features_kwargs)
# preprocessor.encode_sentence_pair('In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.','Current academic and industrial research is interested in autonomous vehicles.')
preprocessor.encode_sentence('In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.')

'W_1.05 C_0.95 L_0.75 WR_0.85 DTD_0.85 In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.'

In [23]:
simple_sentence='Current academic and industrial research is interested in autonomous vehicles.'
complex_sentence='In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.'
round(min(safe_division(get_lexical_complexity_score(simple_sentence),
                                       get_lexical_complexity_score(complex_sentence)),2))

'0.88'

simple_sentence='Current academic and industrial research is interested in autonomous vehicles.'
complex_sentence='In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.'
round(min(safe_division(get_complexity_score(simple_sentence),
                                       get_complexity_score(complex_sentence)), 2))

In [24]:
new_df = preprocessor.preprocess_dataset(df)

100%|████████████████████████████████████████████████████████████████████████████████| 648/648 [00:15<00:00, 40.53it/s]


In [25]:
new_df

,source_snt,simplified_snt
0,W_0.44 C_0.54 L_0.47 WR_0.88 DTD_0.80 In the m...,Current academic and industrial research is in...
1,W_0.44 C_0.44 L_0.52 WR_1.00 DTD_0.36 With the...,Drones are increasingly used in the civilian a...
2,W_0.77 C_0.78 L_0.73 WR_0.98 DTD_0.75 Due to g...,Governments set guidelines on the operation ce...
3,W_0.51 C_0.61 L_0.61 WR_1.00 DTD_0.58 In an at...,Researchers propose data-driven solutions allo...
4,W_0.64 C_0.67 L_0.51 WR_0.96 DTD_0.45 Derived ...,"The algorithm, based on the Inception model, d..."
...,...,...
643,W_0.38 C_0.37 L_0.54 WR_1.00 DTD_0.50 Bodybuil...,Bodybuilders train with moderate loads and sho...
644,W_0.62 C_0.57 L_0.73 WR_0.98 DTD_0.67 Powerlif...,"Powerlifters, on the other hand, train with hi..."
645,W_0.79 C_0.77 L_0.81 WR_0.95 DTD_1.00 Although...,Although both groups are known to display impr...
646,W_0.85 C_0.75 L_0.78 WR_0.93 DTD_0.75 It has b...,It has been shown that many factors mediate th...


## Test data

In [229]:
# test_l=load_data('test/simpletext-task3-test-large.json')
# test_m=load_data('test/simpletext-task3-test-medium.json')
test_s=load_data('test/simpletext-task3-test-small.json')
def load_test_set(file):
    dataset=pd.DataFrame(file,dtype='object')
    for index, row in dataset.iterrows():
        if not pd.isnull(row['query_text']):
            dataset.at[index, 'source_snt'] = row['source_snt'].replace('.', ',') + ' related to ' + row['query_text'] + '.'
    dataset=dataset[['source_snt']]
    return dataset
test_small=load_test_set(test_s)
# test_medium=load_test_set(test_m)
# test_large=load_test_set(test_l)

In [26]:
# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="left"),
        Column("target_text", justify="left"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# Setting up the device for GPU usage
device = 'cuda' if cuda.is_available() else 'cpu'

## Dataset

In [27]:
class TrainDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        row_source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(row_source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_text":row_source_text,
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_mask": target_mask.to(dtype=torch.long),
        }

In [28]:
class TestDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, source_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            source_text (str): column name of source text

        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.source_text = self.data[source_text]
        
    def __len__(self):
        """returns the length of dataframe"""

        return len(self.source_text)

    def __getitem__(self, index):
        """return the input ids, attention masks"""

        source_text = str(self.source_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
        }

## Train, validation and test

In [29]:
# training logger to log training progress
def training_logger_init():
    training_logger = Table(
        Column("Epoch", justify="center"),
        Column("Steps", justify="center"),
        Column("Loss", justify="center"),
        Column("ROUGE_Loss", justify="center"),
        Column("ROUGE_1", justify="center"),
        Column("ROUGE_2", justify="center"),
        Column("ROUGE_L", justify="center"),
        Column("SARI", justify="center"),
        Column("BLEU", justify="center"),
        Column("FKGL", justify="center"),
        title="Training Status",
        pad_edge=False,
        box=box.ASCII,
    )
    return training_logger

# training logger to log training progress
def epoch_training_logger_init():
    epoch_training_logger = Table(
        Column("Epoch", justify="center"),
        Column("Train_Loss", justify="center"),
        Column("Valid_Loss", justify="center"),
        Column("ROUGE_1", justify="center"),
        Column("ROUGE_2", justify="center"),
        Column("ROUGE_L", justify="center"),
        Column("SARI", justify="center"),
        Column("BLEU", justify="center"),
        Column("FKGL", justify="center"),
        title="Training Epoch Status",
        pad_edge=False,
        box=box.ASCII,
    )
    return epoch_training_logger

In [30]:
from rouge import Rouge
from evaluate import load
from nltk.translate.bleu_score import sentence_bleu

def calculate_rouge(predicted, target):
    rouger = Rouge()
    scores = rouger.get_scores(predicted, target, avg=True)
    return scores['rouge-1']['f'],scores['rouge-2']['f'],scores['rouge-l']['f']

def compute_bleu(predicted, target):
    bleu_scores = []
    for i in range(len(predicted)):
        bleu_scores.append(sentence_bleu([target[i]],predicted[i], weights=(0.25, 0.25, 0.25, 0.25)))
    return np.mean(bleu_scores)

def compute_sari(sources, predicted, target):
    sari = load("sari")
    sari_scores=sari.compute(sources=sources, predictions=predicted, references=[[i] for i in target])
    return sari_scores['sari']

sources=["About 95 species are currently accepted.","About 95 species are currently accepted."]
predictions=["About 95 species are currently accepted.","About 95 species are currently accepted."]
references=["About 95 species are currently known.","About 95 species are currently known."]
compute_bleu(predictions,references)
# compute_sari(sources,predictions,references)

0.7721102818691421

In [31]:
from evaluation.sari import corpus_sari
from evaluation.bleu import corpus_bleu
from evaluation.fkgl import corpus_fkgl
sources=["About 95 species are currently accepted.","About 95 species are currently accepted."]
predictions=["About 95 species are currently accepted.","About 95 species are currently accepted."]
references=["About 95 species are currently known.","About 95 species are currently known."]
corpus_sari(sources,predictions,[references],lowercase=True)
corpus_bleu(predictions,[references],lowercase=True)
# corpus_fkgl(sources)

64.34588841607616

In [32]:
def train(epoch, tokenizer, model, device, loader, optimizer,model_params):
    model.train()
    total_loss=[]
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)
        target_mask = data["target_mask"].to(device, dtype=torch.long)[:, 1:]
        source_text = data["source_text"]

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
            decoder_attention_mask=target_mask,
        )
        loss = outputs[0]
        
        generated_ids = model.generate(
                  input_ids = ids,
                  attention_mask = mask, 
                  max_length=model_params["MAX_TARGET_TEXT_LENGTH"], 
                  num_beams=model_params["NUM_BEAMS"],
                  repetition_penalty=model_params["REPETITION_PENALTY"], 
                  length_penalty=model_params["LENGTH_PENALTY"], 
                  early_stopping=True,
                  do_sample=False,
                  temperature=0.25,
                  top_k=120,
                  top_p=0.98,
                  )

        # calculate rouge score
        preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
        target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
        rouge1,rouge2,rougeL = calculate_rouge(preds, target)
#         bleu_score = compute_bleu(preds, target)
#         sari_score = compute_sari(source_text,preds, target)
        sari_score=corpus_sari(source_text,preds, [target],lowercase=False)
        bleu_score=corpus_bleu(preds, [target],lowercase=False)
        fkgl_score=corpus_fkgl(preds)

        # add rouge loss to total loss
        rouge_loss = 1.0 - np.mean([rouge1,rouge2,rougeL])
        complexity_score = get_lexical_complexity_score_batch(target,preds)#higher is better
        
        lambda_ = 0.7
        loss = lambda_ * loss + (1-lambda_)*(1-complexity_score)

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss),str(rouge_loss),str(rouge1),str(rouge2),str(rougeL),str(sari_score),str(bleu_score),str(fkgl_score))
            console.print(training_logger)

        optimizer.zero_grad()
#         (loss + rouge_loss).backward()
        loss.backward()
        optimizer.step()

        total_loss.append(loss)
    average_loss=sum(total_loss)/len(total_loss)

    return average_loss

In [33]:
def validate(tokenizer, model, device, loader,model_params):

    """
    Function to evaluate model for predictions

    """
    model.eval()
    predictions = []
    actuals = []
    total_loss=[]
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []
    sari_scores = []
    fkgl_scores = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data["target_ids"].to(device, dtype=torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data["source_ids"].to(device, dtype=torch.long)
            mask = data["source_mask"].to(device, dtype=torch.long)
            target_mask = data["target_mask"].to(device, dtype=torch.long)[:, 1:]
            source_text = data["source_text"]
            
            outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
            decoder_attention_mask=target_mask,
            )
            loss = outputs[0]

            generated_ids = model.generate(
                  input_ids = ids,
                  attention_mask = mask, 
                  max_length=model_params["MAX_TARGET_TEXT_LENGTH"], 
                  num_beams=model_params["NUM_BEAMS"],
                  repetition_penalty=model_params["REPETITION_PENALTY"], 
                  length_penalty=model_params["LENGTH_PENALTY"], 
                  early_stopping=True,
                  do_sample=False,
                  temperature=0.25,
                  top_k=120,
                  top_p=0.98,
                  )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            rouge1,rouge2,rougeL = calculate_rouge(preds, target)
            sari_score=corpus_sari(source_text,preds, [target],lowercase=False)
            bleu_score=corpus_bleu(preds, [target],lowercase=False)
            fkgl_score=corpus_fkgl(preds)
#             bleu_score = compute_bleu(preds, target)
#             sari_score = compute_sari(source_text,preds, target)

            predictions.extend(preds)
            actuals.extend(target)
            
            total_loss.append(loss)
            rouge1_scores.append(rouge1)
            rouge2_scores.append(rouge2)
            rougeL_scores.append(rougeL)
            sari_scores.append(sari_score)
            bleu_scores.append(bleu_score)
            fkgl_scores.append(fkgl_score)
            
    average_loss=sum(total_loss)/len(total_loss)
    average_rouge1=sum(rouge1_scores)/len(rouge1_scores)
    average_rouge2=sum(rouge2_scores)/len(rouge2_scores)
    average_rougeL=sum(rougeL_scores)/len(rougeL_scores)
    average_bleu=sum(bleu_scores)/len(bleu_scores)
    average_sari=sum(sari_scores)/len(sari_scores)
    average_fkgl=sum(fkgl_scores)/len(fkgl_scores)
    return predictions, actuals,average_loss, average_rouge1,average_rouge2,average_rougeL,average_bleu,average_sari,average_fkgl

In [34]:
def testing(tokenizer, model, device, loader,model_params):

    """
    Function to test model for predictions

    """
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=model_params["MAX_TARGET_TEXT_LENGTH"], 
                num_beams=model_params["NUM_BEAMS"],
                repetition_penalty=model_params["REPETITION_PENALTY"], 
                length_penalty=model_params["LENGTH_PENALTY"], 
                early_stopping=True,
                do_sample=False,
                temperature=0.25,
                top_k=120,
                top_p=0.98,
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            if _%10==0:
                console.print(f'Completed {_}')

            predictions.extend(preds)
    return predictions

## ModelTrainer and ModelTest

In [35]:
def ModelTrainer(
    dataframe, source_text, target_text, model,tokenizer,model_params, output_dir
):

    """
    T5 trainer

    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")
    model = model.to(device)

     # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    train_dataset["source_snt"] = "simplify: " + train_dataset["source_snt"]
    train_dataset = train_dataset.reset_index(drop=True)
    
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    if model_params["CONTROL_TOKENS"]:
        val_dataset["source_snt"]=[preprocessor.encode_sentence(i[38:]) for i in val_dataset["source_snt"]]
    val_dataset["source_snt"] = "simplify: " + val_dataset["source_snt"]
    
    display_df(train_dataset.head(2))

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"VALID Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = TrainDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = TrainDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.AdamW(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train_loss =train(epoch, tokenizer, model, device, training_loader, optimizer,model_params)
        predictions, actuals, valid_loss, average_rouge1,average_rouge2,average_rougeL,average_bleu,average_sari,average_fkgl = validate(tokenizer, model, device, val_loader,model_params)
        epoch_training_logger.add_row(str(epoch), str(train_loss), str(valid_loss), str(average_rouge1),str(average_rouge2),str(average_rougeL),str(average_sari),str(average_bleu),str(average_fkgl))
        console.print(epoch_training_logger)
        
    final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
    final_df.to_csv(os.path.join(output_dir, "valid_predictions.csv"))

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'valid_predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [36]:
def ModelTest(
   test,size,source_text,model,tokenizer,model_params, output_dir
):
    """
    T5 test

    """        
    # logging
    console.log(f"""[Model]: Testing {model_params["MODEL"]}...\n""")
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading {size} test data...\n")

    # Creation of Dataset and Dataloader
    test = test.reset_index(drop=True)
    console.print(f"Test {size} Dataset: {test.shape}")

    # Creating the Training and Validation dataset for further creation of Dataloader
    test_set = TestDataSetClass(
        test,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        source_text,
    )

    # Defining the parameters for creation of dataloaders
    test_params = {
        "batch_size": model_params["TEST_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }
    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    test_loader = DataLoader(test_set, **test_params)

    # Testing loop
    console.log(f"[Initiating Testing]...\n")
    predictions = testing(tokenizer, model, device, test_loader,model_params)
    final_df = pd.DataFrame({"simplified_snt": predictions})
    final_df.to_csv(os.path.join(output_dir, f"test_{size}_predictions.csv"))

    console.log(f"[Testing Completed.]\n")
    console.print(
        f"""[Test] Generation on Test data saved @ {os.path.join(output_dir,f'test_{size}_predictions.csv')}\n"""
    )

In [37]:
def test(test_set,model,tokenizer,size,output_dir):
    test_set=test_set.copy()
    if model_params["CONTROL_TOKENS"]:
        test_set["source_snt"]=[preprocessor.encode_sentence(i) for i in test_set["source_snt"]]
    test_set["source_snt"] = "simplify: " + test_set["source_snt"]
    ModelTest(
        test=test_set,
        source_text="source_snt",
        size=size,
        model=model,
        tokenizer=tokenizer,
        model_params=model_params,
        output_dir=output_dir,
    )

## Load model

In [38]:
def load_model(output_dir,model_class,tokenizer_class,model_name):

    # 获取tokenizer和模型配置信息
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model_config = model_class.config_class.from_pretrained(model_name)

    # 拼接完整路径
    model_path = os.path.join(output_dir, "model_files")

    # 加载模型
    model = model_class.from_pretrained(model_path, config=model_config)

    return model, tokenizer

def generate_summary(text,model,tokenizer):
    if model_params["CONTROL_TOKENS"]:
        text=preprocessor.encode_sentence(text)
    input_ids = tokenizer.encode("simplify: "+text, return_tensors='pt', 
                                    max_length=tokenizer.model_max_length, 
                                    truncation=True)
    summary_ids = model.generate(input_ids, 
                                 max_length=model_params["MAX_TARGET_TEXT_LENGTH"], 
                                 num_beams=model_params["NUM_BEAMS"],
                                 repetition_penalty=model_params["REPETITION_PENALTY"], 
                                 length_penalty=model_params["LENGTH_PENALTY"],
                                 do_sample=False,
                                 temperature=0.25,
                                 top_k=120,
                                 top_p=0.98,)

    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(summary)

## T5 model with control tokens

### T5 Training loop

In [50]:
# tokenzier for encoding the text
model_params = {
    "MODEL": "t5-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TEST_BATCH_SIZE": 8,  # test batch size
    "TRAIN_EPOCHS": 5,  # number of training epochs
    "LEARNING_RATE": 3e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 100,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 75,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
    "NUM_BEAMS":8,
    "REPETITION_PENALTY":2.5, 
    "LENGTH_PENALTY":1, 
    "CONTROL_TOKENS":True,
}

# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
tokenizer = T5TokenizerFast.from_pretrained(model_params["MODEL"])
training_logger = training_logger_init()
epoch_training_logger=epoch_training_logger_init()

# control tokens
features_kwargs = {
        'WordRatioFeature': {'target_ratio': '1.05'},
        'CharRatioFeature': {'target_ratio': '0.95'},
        'LevenshteinRatioFeature': {'target_ratio': '0.75'},
        'WordRankRatioFeature': {'target_ratio': '0.95'},
        'DependencyTreeDepthRatioFeature': {'target_ratio': '0.85'}
    }
# features_kwargs = {}
preprocessor = Preprocessor(features_kwargs)

ModelTrainer(
    dataframe=new_df,
    source_text="source_snt",
    target_text="simplified_snt",
    model=model,
    tokenizer=tokenizer,
    model_params=model_params,
    output_dir="T5_outputs",
)

[06:34:41] [Model]: Loading t5-base...                                           <ipython-input-35-ea8cec386a2b>:17
                                                                                                                   

           [Data]: Reading data...                                               <ipython-input-35-ea8cec386a2b>:21
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|source_text                                             | target_text                                            |
|--------------------------------------------------------+--------------------------------------------------------|
|simplify: W_0.67 C_0.64 L_0.59 WR_0.97 DTD_0.64         | Analysis of studies with different training volumes    |
|Meta-regression analysis of non-volume-equated studies  | showed better results for higher frequencies, although |
|showed a significant effect favoring higher             | the difference between frequencies of 1 and 3+ days per|
|frequencies, although the overall difference in         | week was small.                                        |
|magnitude of effect between frequencies of 1 and 3+     |                                                        |
|days per week was modest, related to How many training  |                                                        |
|per week for hypetrophy?.                               |                                                        |
|simplify: W_0.78 C_0.76 L_0.86 WR_1.06 DTD_0.70 Four    | Four major capabilities were identified, each of which |
|major capabilities were identified, each of which       | evolves as a result of using the tools.                |
|evolves as a result of using the tools, related to      |                                                        |
|digital marketing.                                      |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (648, 2)

TRAIN Dataset: (518, 2)

...

                                               Training Epoch Status                                               
+-----------------------------------------------------------------------------------------------------------------+
|Epoch | Train_Loss | Valid_Loss |  ROUGE_1   |   ROUGE_2   |  ROUGE_L   |    SARI     |    BLEU    |     FKGL    |
|------+------------+------------+------------+-------------+------------+-------------+------------+-------------|
|  0   | tensor(1.… | tensor(0.… | 0.7076091… | 0.58487038… | 0.6965875… | 55.3076671… | 52.048508… | 11.89874514…|
|      | grad_fn=<… |            |            |             |            |             |            |             |
|  1   | tensor(0.… | tensor(0.… | 0.7292614… | 0.62401543… | 0.7190145… | 60.8534967… | 56.014940… | 11.89874514…|
|      | grad_fn=<… |            |            |             |            |             |            |             |
|  2   | tensor(0.… | tensor(0.… | 0.7796764… | 0.69249895… | 0.7713730… | 68.5571183… | 63.955833… | 11.89874514…|
|      | grad_fn=<… |            |            |             |            |             |            |             |
|  3   | tensor(0.… | tensor(0.… | 0.7874813… | 0.71606892… | 0.7776798… | 71.7571479… | 65.515074… | 11.89874514…|
|      | grad_fn=<… |            |            |             |            |             |            |             |
|  4   | tensor(0.… | tensor(0.… | 0.8045090… | 0.73485009… | 0.7958336… | 73.5445415… | 66.925911… | 11.89874514…|
|      | grad_fn=<… |            |            |             |            |             |            |             |
+-----------------------------------------------------------------------------------------------------------------+

[11:33:10] [Saving Model]...                                                     <ipython-input-35-ea8cec386a2b>:96
                                                                                                                   

[Model] Model saved @ T5_outputs\model_files

[Validation] Generation on Validation data saved @ T5_outputs\valid_predictions.csv

[Logs] Logs saved @ T5_outputs\logs.txt

### T5 Testing

In [31]:
model,tokenizer=load_model("T5_outputs",T5ForConditionalGeneration,T5Tokenizer,"t5-small")
test(test_small,model,tokenizer,'small',"T5_outputs")
# test(test_medium,model,tokenizer,'medium',"T5_outputs")
# test(test_medium,model,tokenizer,'large',"T5_outputs")

[19:02:36] [Model]: Testing t5-small...                                           <ipython-input-20-d3131938de95>:9
                                                                                                                   

           [Data]: Reading small test data...                                    <ipython-input-20-d3131938de95>:13
                                                                                                                   

Test small Dataset: (2234, 1)

           [Initiating Testing]...                                               <ipython-input-20-d3131938de95>:37
                                                                                                                   

Completed 0

...

Completed 250

Completed 260

Completed 270

[19:38:01] [Testing Completed.]                                                  <ipython-input-20-d3131938de95>:42
                                                                                                                   

[Test] Generation on Test data saved @ T5_outputs/test_small_predictions.csv

In [54]:
# model,tokenizer=load_model("T5_outputs",T5ForConditionalGeneration,T5TokenizerFast,"t5-base")
text='This is moving us to a tipping point and to a crossroads: we must decide between a society in which the actions are determined in a top-down way and then implemented by coercion or manipulative technologies (such as personalized ads and nudging) or a society, in which decisions are taken in a free and participatory way and mutually coordinated.'
generate_summary(text,model,tokenizer)

This is moving us to a tipping point and to a crossroads: we must choose between a society in which the actions are determined top-down by coercion or manipulative technologies (such as personalized ads and nudging) or in which decisions are taken freely and cooperatively coordinated.


In [33]:
def output_format(size, run):
    #load test results
    file=load_data(f'test/simpletext-task3-test-{size}.json')

    df1=pd.read_csv(f"T5_outputs/test_{size}_predictions.csv")
    df2=pd.DataFrame(file)
    snt_id=df2[['snt_id']]
    
    df1 = df1.drop(df1.columns[0], axis=1)
    df1.insert(0, 'run_id', f'QH_task_3_run{run}')
    df1.insert(1, 'manual', 0)
    df1.insert(2, 'snt_id', snt_id)

    df1.to_json(f'{size}_pre_with_id.json', orient='records')

In [34]:
run = 1
for size in ['small']:
    output_format(size, run)

## T5 model without control tokens

In [ ]:
# tokenzier for encoding the text
model_params = {
    "MODEL": "t5-small",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TEST_BATCH_SIZE": 8,  # test batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 70,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 55,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
    "NUM_BEAMS":8,
    "REPETITION_PENALTY":2.5, 
    "LENGTH_PENALTY":1, 
    "CONTROL_TOKENS":False,
}

# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
tokenizer = T5TokenizerFast.from_pretrained(model_params["MODEL"])
training_logger = training_logger_init()
epoch_training_logger=epoch_training_logger_init()

# features_kwargs = {}
preprocessor = Preprocessor(features_kwargs)

ModelTrainer(
    dataframe=df,
    source_text="source_snt",
    target_text="simplified_snt",
    model=model,
    tokenizer=tokenizer,
    model_params=model_params,
    output_dir="T5_outputs",
)

## BART model

### BART training

In [33]:
# tokenzier for encoding the text
model_params = {
    "MODEL": 'facebook/bart-large-cnn',  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TEST_BATCH_SIZE": 8,  # test batch size
    "TRAIN_EPOCHS": 1,  # number of training epochs
    "LEARNING_RATE": 1e-5,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 100,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 75,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
    "NUM_BEAMS":4,
    "REPETITION_PENALTY":2.5, 
    "LENGTH_PENALTY":0.75, 
}

from transformers import BartTokenizer, BartForConditionalGeneration

# 加载BART模型和分词器
model = BartForConditionalGeneration.from_pretrained(model_params["MODEL"])
tokenizer = BartTokenizer.from_pretrained(model_params["MODEL"])
model.model.decoder.generation_mode=False

training_logger = training_logger_init()
epoch_training_logger=epoch_training_logger_init()

train_data=df.copy()
train_data["source_snt"] = tokenizer.bos_token + train_data["source_snt"] + tokenizer.eos_token
ModelTrainer(
    dataframe=train_data,
    source_text="source_snt",
    target_text="simplified_snt",
    model=model,
    tokenizer=tokenizer,
    model_params=model_params,
    output_dir="Bart_outputs",
)

[06:54:22] [Model]: Loading facebook/bart-large-cnn...                           <ipython-input-27-deeb963aca67>:17
                                                                                                                   

           [Data]: Reading data...                                               <ipython-input-27-deeb963aca67>:21
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|   <s>In the modern era of automation and robotics,     | Current academic and industrial research is interested |
|autonomous vehicles are currently the focus of academic |                 in autonomous vehicles.                |
|    and industrial research, related to drones.</s>     |                                                        |
| <s>With the ever increasing number of unmanned aerial  |    Drones are increasingly used in the civilian and    |
|vehicles getting involved in activities in the civilian |      commercial domain and need to be autonomous.      |
| and commercial domain, there is an increased need for  |                                                        |
| autonomy in these systems too, related to drones.</s>  |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (648, 2)

TRAIN Dataset: (518, 2)

...

                                               Training Epoch Status                                               
+-----------------------------------------------------------------------------------------------------------------+
|Epoch |  Train_Loss  |  Valid_Loss  |   ROUGE_1    |   ROUGE_2    |    ROUGE_L    |     SARI     |      BLEU     |
|------+--------------+--------------+--------------+--------------+---------------+--------------+---------------|
|  0   | tensor(1.48… | tensor(1.29… | 0.469237191… | 0.311362227… | 0.4520826321… | 51.84726098… | 0.35215151439…|
|      | grad_fn=<Di… |              |              |              |               |              |               |
+-----------------------------------------------------------------------------------------------------------------+

[07:57:56] [Saving Model]...                                                     <ipython-input-27-deeb963aca67>:90
                                                                                                                   

[Model] Model saved @ Bart_outputs\model_files

[Validation] Generation on Validation data saved @ Bart_outputs\valid_predictions.csv

[Logs] Logs saved @ Bart_outputs\logs.txt

### Bart Testing

In [ ]:
# model,tokenizer=load_model("Bart_outputs")
model,tokenizer=load_model("Bart_outputs",BartForConditionalGeneration,BartTokenizer,'facebook/bart-large-cnn')
test(test_small,model,tokenizer,'small',"Bart_outputs")
test(test_medium,model,tokenizer,'medium',"Bart_outputs")
test(test_medium,model,tokenizer,'large',"Bart_outputs")

In [36]:
model,tokenizer=load_model("Bart_outputs",BartForConditionalGeneration,BartTokenizer,'facebook/bart-large-cnn')
text='<s>In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research, related to drones.</s>'
generate_summary(text,model,tokenizer)

In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research. It is an important field of study because drones can be automated or robotically controlled. However, in the industrial and academic studies, people with human form of control are not allowed to drive autonomous vehicles.


## FlaxT5 model

### Flax T5 training

In [40]:
# tokenzier for encoding the text
model_params = {
    "MODEL": 'google/t5-v1-1.1.0',  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TEST_BATCH_SIZE": 8,  # test batch size
    "TRAIN_EPOCHS": 1,  # number of training epochs
    "LEARNING_RATE": 3e-5,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 100,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 75,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
    "NUM_BEAMS":4,
    "REPETITION_PENALTY":2.5, 
    "LENGTH_PENALTY":0.75, 
}

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 加载预训练的GPT-2模型和tokenizer
from transformers import FlaxT5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = FlaxT5ForConditionalGeneration.from_pretrained(model_name)

training_logger = training_logger_init()
epoch_training_logger=epoch_training_logger_init()


ModelTrainer(
    dataframe=train_data,
    source_text="source_snt",
    target_text="simplified_snt",
    model=model,
    tokenizer=tokenizer,
    model_params=model_params,
    output_dir="GPT2_outputs",
)

[10:51:48] [Model]: Loading gpt2...                                              <ipython-input-27-deeb963aca67>:17
                                                                                                                   

           [Data]: Reading data...                                               <ipython-input-27-deeb963aca67>:21
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|     In the modern era of automation and robotics,      | Current academic and industrial research is interested |
|autonomous vehicles are currently the focus of academic |                 in autonomous vehicles.                |
|      and industrial research, related to drones.       |                                                        |
|  With the ever increasing number of unmanned aerial    |    Drones are increasingly used in the civilian and    |
|vehicles getting involved in activities in the civilian |      commercial domain and need to be autonomous.      |
| and commercial domain, there is an increased need for  |                                                        |
|   autonomy in these systems too, related to drones.    |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (648, 2)

TRAIN Dataset: (518, 2)

TEST Dataset: (130, 2)

           [Initiating Fine Tuning]...                                           <ipython-input-27-deeb963aca67>:79
                                                                                                                   

TypeError: forward() got an unexpected keyword argument 'decoder_input_ids'